# Virtual experiments, replicating Lahiri and Bevan 2020

In this notebook, we try and replicate a few of the dopamine experiments in Lahiri and Bevan 2020.

4Hz optogenetical stimulation (5 pulses) yielded approximately 0.3 uM DA concentration, while 20Hz stimulation yielded (5 pulses) on average 0.5 uM DA concentration. Here we model the optogenetic activation by setting the DA concentration to these values. In both cases the duration of the resulting DA pulse was set to 5 seconds. 


## Experiment 1

Current injection driving the dSPN to approximately 10Hz, after 5 seconds 0.3 uM DA is added, then current injection is active for another 5 seconds.

Compare to Figure 1E, 1G, 1H

In [1]:
import os
from snudda import Snudda

neuron = "dspn"
# neuron = "ispn"

neuron_path = os.path.join("..", "..", "snudda_data", "neurons", "striatum", neuron)
network_path = os.path.join("networks", "bevan_fig1g_bath_current_short")

In [2]:
snudda = Snudda(network_path=network_path)
si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names=neuron, number_of_neurons=[5], 
                      random_seed=1234)

si.network_data["regions"]["Cube"]["neurons"]["dspn"]["reaction_diffusion"] = "../../data/JSON/reaction_diffusion_D1_from_SBTab.json"
# si.network_data["regions"]["Cube"]["neurons"]["ispn"]["reaction_diffusion"] = "data/JSON/reaction_diffusion_D2.json-updated"


# How the ion channels are modified by DA
# OBS, we include SK direkt modulation, in relality it should be modulated by DA acting on Ca 
# si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "modulation_parameters.json"
si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation"] = "../../data/JSON/modulation_parameters-v2.json"

si.network_data["regions"]["Cube"]["neurons"][neuron]["modulation_key"] = "abc"

si.write_json()

snudda.create_network()

Warning, no snudda_data set!
Adding neurons: dspn from dir ../../snudda_data/neurons/striatum/dspn
Writing networks/bevan_fig1g_bath_current_short/network-config.json
Writing networks/bevan_fig1g_bath_current_short/network-config.json
Placing neurons
Network path: networks/bevan_fig1g_bath_current_short
Creating missing directory networks/bevan_fig1g_bath_current_short/log
Created directory networks/bevan_fig1g_bath_current_short/log
Reading SNUDDA_DATA=None from networks/bevan_fig1g_bath_current_short/network-config.json
No n_putative_points and putative_density, setting n_putative_points = 102
(this must be larger than the number of neurons you want to place)
Generating 102 points for networks/bevan_fig1g_bath_current_short/mesh/Cube-cube-mesh-3.9602691983237216e-05.obj
Filtering, keeping inside points: 8 / 44
neuron_name = 'dspn_0', num = np.int64(1), neuron_path = '../../snudda_data/neurons/striatum/dspn/str-dspn-e150602_c1_D1-mWT-0728MSN01-v20220620'
neuron_name = 'dspn_1', num = 

In [3]:
# Free memory
snudda = None

mech_dir = "/home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms"
sample_dt = None # 0.00005

sim_config_on4 = os.path.join("..", "..", "data", "JSON", "bevan_fig1g_with_DA_4Hz_stim_short.json")
sim_config_on20 = os.path.join("..", "..", "data", "JSON", "bevan_fig1g_with_DA_20Hz_stim_short.json")

sim_config_off = os.path.join("..", "..", "data", "JSON", "bevan_fig1g_no_DA_short.json")

sim_output_neuromodulation_ON4 = os.path.join(network_path, "simulation", "output_neuromodulation_ON_4Hz.hdf5")
sim_output_neuromodulation_ON20 = os.path.join(network_path, "simulation", "output_neuromodulation_ON_20Hz.hdf5")

sim_output_neuromodulation_OFF = os.path.join(network_path, "simulation", "output_neuromodulation_OFF.hdf5")

sim_time = 2.2
n_workers = 5

In [ ]:
!nrnivmodl ../../snudda_data/neurons/mechanisms/

Please run the following 3 lines in a terminal

In [4]:
# sim_time=0.1  # TEMP to test!
run_str_on4 = f"mpirun -n {n_workers} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_on4} --mechdir {mech_dir} --enable_rxd_neuromodulation"
print(run_str_on4)
run_str_on20 = f"mpirun -n {n_workers} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_on20} --mechdir {mech_dir} --enable_rxd_neuromodulation"
print(run_str_on20)
run_str_off = f"mpirun -n {n_workers} snudda simulate {network_path} --time {sim_time} --simulation_config {sim_config_off} --mechdir {mech_dir} "
print(run_str_off)

mpirun -n 5 snudda simulate networks/bevan_fig1g_bath_current_short --time 2.2 --simulation_config ../../data/JSON/bevan_fig1g_with_DA_4Hz_stim_short.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms --enable_rxd_neuromodulation
mpirun -n 5 snudda simulate networks/bevan_fig1g_bath_current_short --time 2.2 --simulation_config ../../data/JSON/bevan_fig1g_with_DA_20Hz_stim_short.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms --enable_rxd_neuromodulation
mpirun -n 5 snudda simulate networks/bevan_fig1g_bath_current_short --time 2.2 --simulation_config ../../data/JSON/bevan_fig1g_no_DA_short.json --mechdir /home/hjorth/HBP/BasalGangliaData/data/neurons/mechanisms 


# Disabled for now, run it in terminal!
os.system(run_str_on4)
os.system(run_str_on20)
os.system(run_str_off)

# Analysing 4Hz DA stim

In [ ]:
from snudda.plotting.plot_simulation_reaction_diffusion import PlotReactionDiffusion

# By passing the sls object we avoid loading it twice
prd = PlotReactionDiffusion(network_path=network_path,
                            simulation_file=sim_output_neuromodulation_ON4)

for i in [0,1]:
    print(f"Neuron {i} has data: {prd.list_neuron_info(i)}\n\n")

In [ ]:
prd.plot(neuron_id=0, species=["D1R_DA", "PKAc", "cAMP"], fig_name="biochem-0-normalised_DA_4Hz.svg", title="dSPN, selected species", normalise=True)

In [ ]:
prd.plot(neuron_id=0, 
         species=['kaf_ms.modulation_factor_g', 'kaf_ms.modulation_factor_shift', 'kas_ms.modulation_factor', 'kir_ms.modulation_factor', 'naf_ms.modulation_factor',
                 'sk_ms.modulation_factor', 'cal12_ms.modulation_factor', 'cal13_ms.modulation_factor', 'car_ms.modulation_factor'], 
         species_label=['kaf (g)', 'kaf (shift)', 'kas (g)', 'kir (g)', 'naf (g)', 'sk (g)', 'cal12 (g)', 'cal13 (g)', 'car (g)'],
         ylabel="Modulation", fig_name="modulation-0_DA_4Hz.svg", title="dSPN modulation", width=800, height=700)

# Analysing 20Hz DA

In [ ]:
from snudda.plotting.plot_simulation_reaction_diffusion import PlotReactionDiffusion

# By passing the sls object we avoid loading it twice
prd = PlotReactionDiffusion(network_path=network_path,
                            simulation_file=sim_output_neuromodulation_ON20)

for i in [0,1]:
    print(f"Neuron {i} has data: {prd.list_neuron_info(i)}\n\n")

In [ ]:
prd.plot(neuron_id=0, species=["D1R_DA", "PKAc", "cAMP"], fig_name="biochem-0-normalised_DA_20Hz.svg", title="dSPN, selected species", normalise=True)

In [ ]:
prd.plot(neuron_id=0, 
         species=['kaf_ms.modulation_factor_g', 'kaf_ms.modulation_factor_shift', 'kas_ms.modulation_factor', 'kir_ms.modulation_factor', 'naf_ms.modulation_factor',
                 'sk_ms.modulation_factor', 'cal12_ms.modulation_factor', 'cal13_ms.modulation_factor', 'car_ms.modulation_factor'], 
         species_label=['kaf (g)', 'kaf (shift)', 'kas (g)', 'kir (g)', 'naf (g)', 'sk (g)', 'cal12 (g)', 'cal13 (g)', 'car (g)'],
         ylabel="Modulation", fig_name="modulation-0_DA_20Hz.svg", title="dSPN modulation", width=800, height=700)

# Spike frequency

In [ ]:
from snudda.utils import SnuddaLoadSimulation

nd_on4 = SnuddaLoadSimulation(sim_output_neuromodulation_ON4)
data_types = nd_on4.list_data_types(0)
all_species_data = nd_on4.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time_on4 = nd_on4.get_time()
voltage_on4 = nd_on4.get_data("voltage")

nd_on20 = SnuddaLoadSimulation(sim_output_neuromodulation_ON20)
data_types = nd_on20.list_data_types(0)
all_species_data = nd_on20.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time_on20 = nd_on20.get_time()
voltage_on20 = nd_on20.get_data("voltage")

nd_off = SnuddaLoadSimulation(sim_output_neuromodulation_OFF)
time_off = nd_off.get_time()
voltage_off = nd_off.get_data("voltage")

In [ ]:
voltage_off[0]

In [ ]:
import numpy as np
import json
with open("../../data/JSON/bevan_fig1g_with_DA_short.json", "r") as f:
    config = json.load(f)

da_time = np.array(config["bath_application"]["DA"]["time"])
da_conc = np.array(config["bath_application"]["DA"]["concentration"]) * 1e6


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import numpy as np

pio.renderers.default = "iframe"

# ---- Compute global voltage limits (shared across voltage plots) ----
all_voltages = []

for neuron_id in voltage_on4[0].keys():
    all_voltages.extend(voltage_on4[0][neuron_id][:, 0] * 1e3)
    all_voltages.extend(voltage_on20[0][neuron_id][:, 0] * 1e3)
    all_voltages.extend(voltage_off[0][neuron_id][:, 0] * 1e3)

vmin = np.min(all_voltages)
vmax = np.max(all_voltages)
padding = 0.05 * (vmax - vmin)
ylims = [vmin - padding, vmax + padding]


for neuron_id in voltage_on4[0].keys():

    fig = make_subplots(
        rows=4,
        cols=1,
        shared_xaxes=True,
        vertical_spacing=0.03,
        subplot_titles=[
            "ON (4 DA)",
            "ON (20 DA)",
            "OFF (No DA)",
            "Dopamine Concentration"
        ]
    )

    # -------- Voltage ON (4 DA) --------
    fig.add_trace(
        go.Scatter(
            x=time_on4,
            y=voltage_on4[0][neuron_id][:, 0] * 1e3,
            line=dict(color="red", width=3),
            opacity=0.8,
            showlegend=False,
        ),
        row=1, col=1
    )

    # -------- Voltage ON (20 DA) --------
    fig.add_trace(
        go.Scatter(
            x=time_on20,
            y=voltage_on20[0][neuron_id][:, 0] * 1e3,
            line=dict(color="darkred", width=3),
            opacity=0.8,
            showlegend=False,
        ),
        row=2, col=1
    )

    # -------- Voltage OFF --------
    fig.add_trace(
        go.Scatter(
            x=time_off,
            y=voltage_off[0][neuron_id][:, 0] * 1e3,
            line=dict(color="black", width=3),
            opacity=0.8,
            showlegend=False,
        ),
        row=3, col=1
    )

    # -------- Dopamine concentration --------
    fig.add_trace(
        go.Scatter(
            x=da_time,
            y=da_conc,
            mode="lines",
            line=dict(width=0),
            fill="tozeroy",
            fillcolor="rgba(255,0,0,0.25)",
            showlegend=False,
        ),
        row=4, col=1
    )

    # ---- Axis formatting ----
    for r in [1, 2, 3]:
        fig.update_yaxes(
            title_text="Voltage (mV)",
            range=ylims,
            ticks="outside",
            tickfont=dict(size=20),
            title_font=dict(size=24),
            row=r, col=1
        )

    fig.update_yaxes(
        title_text="DA (nM)",
        ticks="outside",
        tickfont=dict(size=20),
        title_font=dict(size=24),
        row=4, col=1
    )

    fig.update_xaxes(
        title_text="Time (s)",
        range=[0, 8],
        dtick=2,
        ticks="outside",
        tickfont=dict(size=20),
        title_font=dict(size=24),
        row=4, col=1
    )

    # ---- Global layout (publication style) ----
    fig.update_layout(
        width=1200,
        height=1100,
        font=dict(
            family="Arial",
            size=22,
            color="black"
        ),
        margin=dict(l=100, r=40, t=80, b=80),
        paper_bgcolor="white",
        plot_bgcolor="white"
    )

    fig.show()

    fig.write_image(
        f"bevan-figure-{neuron_id}-volt-trace.png",
        scale=2,
        width=1200,
        height=1100
    )
    fig.write_image(
        f"bevan-figure-{neuron_id}-volt-trace.pdf",
        scale=2,
        width=1200,
        height=1100
    )


In [ ]:
from snudda.plotting.plot_simulation_reaction_diffusion import PlotReactionDiffusion

# By passing the sls object we avoid loading it twice
prd = PlotReactionDiffusion(network_path=network_path,
                            simulation_file=simulation_file,
                            snudda_simulation=sls)

print(f"Neuron {o} has data: {prd.list_neuron_info(0)}\n\n")
prd.plot(neuron_id=0, species=["D1R_DA", "PKAc", "cAMP"], fig_name="biochem-0-normalised.svg", title="dSPN, selected species", normalise=True)

prd.plot(neuron_id=0, 
         species=['kaf_ms.modulation_factor_g', 'kaf_ms.modulation_factor_shift', 'kas_ms.modulation_factor', 'kir_ms.modulation_factor', 'naf_ms.modulation_factor',
                 'sk_ms.modulation_factor', 'cal12_ms.modulation_factor', 'cal13_ms.modulation_factor', 'car_ms.modulation_factor'], 
         species_label=['kaf (g)', 'kaf (shift)', 'kas (g)', 'kir (g)', 'naf (g)', 'sk (g)', 'cal12 (g)', 'cal13 (g)', 'car (g)'],
         ylabel="Modulation", fig_name="modulation-0.svg", title="dSPN modulation", width=800, height=700)

# Plot frequency change plots

In [ ]:
from snudda.plotting import SnuddaPlotSpikeRaster2

network_file = os.path.join(network_path, "network-synapses.hdf5")
spr_on4 = SnuddaPlotSpikeRaster2(network_path=network_path, network_file=network_file, simulation_file=sim_output_neuromodulation_ON4)

network_file = os.path.join(network_path, "network-synapses.hdf5")
spr_on20 = SnuddaPlotSpikeRaster2(network_path=network_path, network_file=network_file, simulation_file=sim_output_neuromodulation_ON20)

network_file = os.path.join(network_path, "network-synapses.hdf5")
spr_off = SnuddaPlotSpikeRaster2(network_path=network_path, network_file=network_file, simulation_file=sim_output_neuromodulation_OFF)

time_ranges = [(1.2, 2.2)]  # For short version, only one time range
neuron_id_all = [0, 1, 2, 3, 4]
freq_on4 = spr_on4.snudda_simulation_load.get_frequency(neuron_id=neuron_id_all, time_ranges=time_ranges)
freq_on20 = spr_on4.snudda_simulation_load.get_frequency(neuron_id=neuron_id_all, time_ranges=time_ranges)
freq_off = spr_off.snudda_simulation_load.get_frequency(neuron_id=neuron_id_all, time_ranges=time_ranges)



In [ ]:
freq_on4, freq_on20, freq_off

## Need to update plots to show data in new way

no f4 f20 
lines drawn between them

In [ ]:
def make_freq_plot(data, title):
    fig = go.Figure()

    # Add one line per row
    for row in data:
        fig.add_trace(go.Scatter(
            x=[0, 1],
            y=row,
            mode='lines+markers',
            line=dict(color='gray', width=2),
            marker=dict(size=10, color='black'),
            showlegend=False
        ))

    # Mean line
    mean_vals = np.mean(data, axis=0)
    fig.add_trace(go.Scatter(
        x=[0, 1],
        y=mean_vals,
        mode='lines+markers',
        line=dict(color='black', width=6),
        marker=dict(size=20, color='black'),
        name='Mean'
    ))

    # Layout (new format)
    fig.update_layout(
        title=dict(
            text=title,
            font=dict(size=40)   # bigger title font if you like
        ),
        xaxis=dict(
            tickmode='array',
            tickvals=[0, 1],
            ticktext=["t=(1,2)", "t=(6,7)"],
            title=dict(text="", font=dict(size=20)),
            tickfont=dict(size=30)
        ),
        yaxis=dict(
            title=dict(text="Frequency (Hz)", font=dict(size=40)),
            tickfont=dict(size=30)
        ),
        width=400,
        height=500,
        font=dict(size=16),
        legend=dict(
            font=dict(size=30)   # adjust as needed
        )
    )
    return fig


In [ ]:
# Create and show plots
fig_on = make_freq_plot(freq_on4, "With Dopamine (4Hz)")
fig_on.show()
fig_on.write_image("fig1_frequency_change_with_da_4hz.pdf", scale=2)
fig_on.write_image("fig1_frequency_change_with_da_4hz.png", scale=2)
#fig_on.write_image("fig1_frequency_change_with_da.pdf", scale=2)

In [ ]:
# Create and show plots
fig_on = make_freq_plot(freq_on20, "With Dopamine (20Hz)")
fig_on.show()
fig_on.write_image("fig1_frequency_change_with_da_20hz.pdf", scale=2)
fig_on.write_image("fig1_frequency_change_with_da_20hz.png", scale=2)
#fig_on.write_image("fig1_frequency_change_with_da.pdf", scale=2)

In [ ]:
fig_off = make_freq_plot(freq_off, "Without Dopamine")
fig_off.show()
fig_off.write_image("fig1_frequency_change_no_da.pdf", scale=2)
fig_off.write_image("fig1_frequency_change_no_da.png", scale=2)
#fig_off.write_image("fig1_frequency_change_no_da.pdf", scale=2)

## Experiment 2

Can we show persistence for 15 minutes? (Compare Figure 2)

## Experiment 3

Current injection for 3 seconds, trigger 2 (4) spikes in dSPN. After DA application (1 second, 0.5u M) spikes come earlier and at a higher frequency.

Compare Figure 4 B, C, D

## Experiment 4

Subthreshold current injection (<10 mV depolarisation). Size of depolarisation not affected by DA.

Suprathreshold current injection, with DA lower threshold, and faster spiking.

Compare Figure 5

In [ ]:
Experiment 5

Compare Figure 6